# Arbitrary Oriented Scene Text Recognition via rotation proposals



Notes on the paper Arbitrary-Oriented Scene Text Detection via Rotation Proposals by Jianqi Ma 


## Network architecture -Rough points 


![arch](arch)

So just a set of bullet points that describe the newtork - 
* You feed the input Image into a VGG network and (as per the keras code ) the final layer in the vgg network has 512 filter and is of the size (3,3). This part is from faster rcnn 


* Here there is a split, in the first arm of the split, the feature map is fed into the RRPN 


* Second arm of the split, the feature map is directly passed onto the RROI pooling layer


* The RROI pooling layer creates three different layers, the first one accounts for the background, the second for the text and the third for background ? is that it ? 


* The RRPN just like faster Rcnn's RPN is minimized by a combined loss function that accounts for both regression and class based classification of the bounding boxes 


* The bottlenecks of understanding are essentially how are the region proposal calucated, what is happening during ROI pooling and RROI pooling, i guess once you understand ROI pooling, it must be easy to genearalize that to RROI pooling



## Input data format 




The format of the input data is a bounding box with the parameters
$$(x,y,w,h, \theta)$$

$ \theta $ has a range where such that  $$ \theta  \rightarrow  \big[ - (1/4)\pi,(3/4)\pi     \big) $$  

here $h$ is the short side of the bounding box and $w$ is the long side of the bounding box 

Orientation parameter is added to control the orientation of a proposal- $$-\pi/6, 0, \pi/6,\pi/3, \pi/2, (2/3)\pi $$

Scales of the bounding box are 2, 5 and 8 


![anchors](rrpn_anchors)


In total for each point in the feature map there are $6 \times 3 \times 3 = 54  \:$ rotational anchors that are generated. There are  $5 \times 54 =270  $ regression outputs and $2 \times 54  = 108 $  class outputs that are generated  


There are, for a feature map of $ (H,W) $ ,  $ H \times W \times 54 \:$  region proposals in total. So one **important question** is how do you connect the feature map to the input image? what does a single point on the feature map mean with respect to the input image ? 


So probably what is happening is that for a single point in the feature map, there is a corresponding region in the image, this is due to max pooling. So we should place the center of the R anchor at the center of this region, but here are some more questions 

* If the above is the case, suppose you have two or more objects in the region, you will have two or more ground truth anchors, what do you do ? if the R-anchor intersects both of them equally what choice do you have ? 
* How do you adjust for the size of the anchor with respect to the ground truth anchor? 
* what is the beginning size of the anchor? i guess that is set on the scale and the ratio of the anchors. It probably is also the issue that the probability that your ground truth anchor is the same size as the predicted anchor size
* during inference how does it know what is the size of the anchor that you must have in order to fit a bbox to the detected object 

## Learning the loss function 

First step to calculate the loss function is to figure out what is the Intersection over union between the predicted anchor and the ground truth anchor

* positive IOU's -value $> 0.7 \:$  AND angle $ <\pi/12$



* negative IOU's -value $< 0.3\:$ or $> 0.7 \:$  AND  angle $ >\pi/12$

so this means that if you have an IOU value  $< 0.7$ but angle is angle $ >\pi/12 \:$  then it will not be considered a an intersection. This will not be used to calculate the loss 

Once an anchors qualifies the above conditions, the loss is calculated, the loss is given by 

$$ \begin{equation} 
     L(p,l,v^{*},v) = {L_{cls}}(p,l) + {L_{reg}}(v^{*},v)   
     \end{equation}
$$

where $v = (v_{x},v_{y},v_{w},v_{h}, v_{\theta})$

The classification loss $$ \begin{equation}{L_{cls}}(p,l) = \log\quad p_l \end {equation} $$


Regression loss is the L1 smooth function 

$$ \begin{equation} {L_{reg}}(v^{*},v) = \sum \text{smooth}_{L1}({v^{*}}_{i} -{v^{}}_{i}) \end {equation} $$  

where $i$ is over all anchors. 

The smooth function is
    
$$\begin{equation} \text{smooth}_{L1}(x) = \begin{cases}
  0.5x^{2} \quad    |x|<1 \\    
      |x| -0.5 \quad otherwise
\end{cases} \end {equation} $$

For the regression loss function we have - 

![eqns](eqns)

where $ a  \ominus  b = a -b + k\pi  $ 

So the next thing to make sense is of RROI pooling. How is that being done ? 

Once an anchor has gone through the RRPN what is happening ?  

* From my understanding of ROI pooling it seems that each region proposal is taken and max pooled. In doing so, regardless of the size of the ROI, the max pooling reduces the ROI to a layer of say for example 7 by 7. If this is the case then what you will have is a stack of layers whose size will be the size of the number of region proposal that are detected.

* What does it mean to max pool the region proposal. The idea is I guess that you are essentially cropping the original image based on the predicted anchor, then you are max pooling this to get the right dimension. This makes sense to a large extend. I guess the max pool does not care if you have a rotated roi. 

* So suppose you have only 1 image then and you get say 10 anchors which match the ground truth bounding box, then you will have $7  \times 7 \times 10$ cropped images after the roi pooling where the 10 channels represents the various predicted anchors 

* Here is another question, each time you are going to have a different number of region proposals the total dimension post roi pooling with be different each time, wont it ? 




## End the network

So what happens at the end of the network? 
* Post roi pooling, you will have a feature map which is the size of the region proposals. 

* You have to feed this into a classifier, essentially in the R-CNN paper they feed it into an SVM, i guess what you can do is feed it into a softmax layer to feed classify whether it an object of interest or not

* Hmmm what if you give it, apart from examples of the object, examples of the background as well so it can do a 1 or 0 classification based on the object or background 

* In our case, since we have a single type of object, its essentially a softmax layer which yields 1 or 0. But this it must do for each region proposal should it not ?

* Since the number of ROI may not remain the same each time you would need a variable number of output neurons, that would not make any sense? 

* From the keras rcnn code it seems that the ** roi pooling is done of the feature map and not on the original image ** . Does that make sense ? 

* In some respects it probably does since the feature maps with its channels will have a lot more information with for a classifier to learn about the problem then just the input image. Since the feature map emerges from the result of training a network on the original image, the feature map is a feature extractor of the image highlighting the important parts of the image.

* I suspect in certain cases it makes sense to scale the feature map up as well. Perhaps one can scale the feature map up, crop the ROI from it and then do the pooling.  